In [1]:
import pandas as pd
import numpy as np
import math

In [2]:
uid_train = pd.read_csv('../../trainData/uid_train.txt',sep='\t',header=None,names=('uid','label'))
voice_train = pd.read_csv('../../trainData/voice_train.txt',sep='\t',header=None,names=('uid','opp_num','opp_head','opp_len','start_time','end_time','call_type','in_out'),dtype={'start_time':str,'end_time':str})
sms_train = pd.read_csv('../../trainData/sms_train.txt',sep='\t',header=None,names=('uid','opp_num','opp_head','opp_len','start_time','in_out'),dtype={'start_time':str})
wa_train = pd.read_csv('../../trainData/wa_train.txt',sep='\t',header=None,names=('uid','wa_name','visit_cnt','visit_dura','up_flow','down_flow','wa_type','date'),dtype={'date':str})

/home/xuegsh/anaconda3/lib/python3.6/site-packages/IPython/core/interactiveshell.py:2728: DtypeWarning: Columns (2) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


In [3]:
voice_test = pd.read_csv('../../testData_B/voice_test_b.txt',sep='\t',header=None,names=('uid','opp_num','opp_head','opp_len','start_time','end_time','call_type','in_out'),dtype={'start_time':str,'end_time':str})
sms_test = pd.read_csv('../../testData_B/sms_test_b.txt',sep='\t',header=None,names=('uid','opp_num','opp_head','opp_len','start_time','in_out'),dtype={'start_time':str})
wa_test = pd.read_csv('../../testData_B/wa_test_b.txt',sep='\t',header=None,names=('uid','wa_name','visit_cnt','visit_dura','up_flow','down_flow','wa_type','date'),dtype={'date':str})

In [4]:
uid_test = pd.DataFrame({'uid':pd.unique(wa_test['uid'])})

In [5]:
voice = pd.concat([voice_train,voice_test],axis=0)
sms = pd.concat([sms_train,sms_test],axis=0)
wa = pd.concat([wa_train,wa_test],axis=0)
uid = pd.concat([uid_train, uid_test], axis=0)

In [6]:
risk_uids_train = uid_train[uid_train['label'] == 1]['uid']
non_risk_uids_train = uid_train[uid_train['label'] == 0]['uid']

# Feat1

In [44]:
voice_train['opp_num'].shape
voice_test['opp_num'].shape
voice_train['opp_num'].unique().shape
voice_test['opp_num'].unique().shape

voice_train[voice_train['opp_num'].isin(voice_test['opp_num'])]['opp_num'].unique().shape


(1150778,)

(609575,)

(221942,)

(122584,)

(6577,)

In [45]:
risk_voice_opp_number = voice_train[voice_train['uid'].isin(risk_uids_train)]['opp_num'].unique()
# risk_voice_opp_number = voice_train[voice_train['opp_num'].isin(maybe_risk_voice_opp_number)]['opp_num'].value_counts()
# # 将与风险用户通话超过10次的号码记为risk_voice_opp_number
# risk_voice_opp_number = risk_voice_opp_number[risk_voice_opp_number > 10].index.values
# risk_voice_opp_number.shape
risk_voice_opp_number.shape

(28180,)

In [46]:
# 统计每一个用户和这些risk_voice_opp_number通话的次数
times = voice[voice['opp_num'].isin(risk_voice_opp_number)].groupby(['uid'])['opp_num'].agg({'voice_count_with_risk': 'count'}).reset_index()
times_all = voice.groupby(['uid'])['opp_num'].agg({'voice_fraction_with_risk': 'count'}).reset_index()

voice_times_with_risk = pd.merge(uid, times, how='left', on='uid')
voice_times_with_risk = pd.merge(voice_times_with_risk, times_all, how='left', on='uid')
voice_times_with_risk['voice_count_with_risk'] = voice_times_with_risk['voice_count_with_risk'].fillna(0)
voice_times_with_risk['voice_fraction_with_risk'] = voice_times_with_risk['voice_count_with_risk'] / voice_times_with_risk['voice_fraction_with_risk']
voice_times_with_risk['voice_fraction_with_risk'] = voice_times_with_risk['voice_fraction_with_risk'].fillna(0)

voice_times_with_risk.sort_values(by='voice_fraction_with_risk', ascending=False)


/home/xuegsh/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:2: FutureWarning: using a dict on a Series for aggregation
is deprecated and will be removed in a future version
  
/home/xuegsh/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:3: FutureWarning: using a dict on a Series for aggregation
is deprecated and will be removed in a future version
  This is separate from the ipykernel package so we can avoid doing imports until


,label,uid,voice_count_with_risk,voice_fraction_with_risk
4494,1.0,u4495,2266.0,1.0
4576,1.0,u4577,16.0,1.0
4567,1.0,u4568,46.0,1.0
4568,1.0,u4569,99.0,1.0
4569,1.0,u4570,311.0,1.0
4570,1.0,u4571,41.0,1.0
4571,1.0,u4572,19.0,1.0
4572,1.0,u4573,227.0,1.0
4573,1.0,u4574,109.0,1.0
4574,1.0,u4575,22.0,1.0


In [47]:
pd.isnull(voice_times_with_risk).any()

label                        True
uid                         False
voice_count_with_risk       False
voice_fraction_with_risk    False
dtype: bool

In [48]:
voice_times_with_risk[voice_times_with_risk['label']==0].sort_values(by=['label', 'voice_fraction_with_risk'], ascending=False).head(50)

,label,uid,voice_count_with_risk,voice_fraction_with_risk
361,0.0,u0362,1.0,1.000000
377,0.0,u0378,1.0,1.000000
500,0.0,u0501,1.0,1.000000
528,0.0,u0529,5.0,1.000000
578,0.0,u0579,1.0,1.000000
826,0.0,u0827,3.0,1.000000
893,0.0,u0894,1.0,1.000000
929,0.0,u0930,2.0,1.000000
930,0.0,u0931,1.0,1.000000
988,0.0,u0989,1.0,1.000000


In [49]:
voice_times_with_risk = voice_times_with_risk.drop(['label'], axis=1)

# Feat2

In [50]:
sms_train['opp_num'].shape
sms_test['opp_num'].shape
sms_train['opp_num'].unique().shape
sms_test['opp_num'].unique().shape

sms_train[sms_train['opp_num'].isin(sms_test['opp_num'])]['opp_num'].unique().shape

(302976,)

(159502,)

(18577,)

(11102,)

(1190,)

In [51]:
risk_sms_opp_number = sms_train[sms_train['uid'].isin(risk_uids_train)]['opp_num'].unique()
risk_sms_opp_number.shape

(3901,)

In [52]:
# 统计每一个用户和这些risk_sms_opp_number通话的次数
times = sms[sms['opp_num'].isin(risk_sms_opp_number)].groupby(['uid'])['opp_num'].agg({'sms_count_with_risk': 'count'}).reset_index()
times_all = sms.groupby(['uid'])['opp_num'].agg({'sms_fraction_with_risk': 'count'}).reset_index()

sms_times_with_risk = pd.merge(uid, times, how='left', on='uid')
sms_times_with_risk = pd.merge(sms_times_with_risk, times_all, how='left', on='uid')
sms_times_with_risk['sms_count_with_risk'] = sms_times_with_risk['sms_count_with_risk'].fillna(0)
sms_times_with_risk['sms_fraction_with_risk'] = sms_times_with_risk['sms_count_with_risk'] / sms_times_with_risk['sms_fraction_with_risk']
sms_times_with_risk['sms_fraction_with_risk'] = sms_times_with_risk['sms_fraction_with_risk'].fillna(0)

sms_times_with_risk.sort_values(by='sms_fraction_with_risk', ascending=False)


/home/xuegsh/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:2: FutureWarning: using a dict on a Series for aggregation
is deprecated and will be removed in a future version
  
/home/xuegsh/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:3: FutureWarning: using a dict on a Series for aggregation
is deprecated and will be removed in a future version
  This is separate from the ipykernel package so we can avoid doing imports until


,label,uid,sms_count_with_risk,sms_fraction_with_risk
4419,1.0,u4420,6.0,1.0
3354,0.0,u3355,2.0,1.0
1357,0.0,u1358,1.0,1.0
3355,0.0,u3356,2.0,1.0
6591,NaN,u8592,22.0,1.0
1353,0.0,u1354,1.0,1.0
4817,1.0,u4818,12.0,1.0
6592,NaN,u8593,1.0,1.0
1350,0.0,u1351,1.0,1.0
4816,1.0,u4817,12.0,1.0


In [53]:
sms_times_with_risk[sms_times_with_risk['label']==0].sort_values(by=['label', 'sms_fraction_with_risk'], ascending=False).head(500)

,label,uid,sms_count_with_risk,sms_fraction_with_risk
22,0.0,u0023,2.0,1.000000
50,0.0,u0051,3.0,1.000000
55,0.0,u0056,9.0,1.000000
76,0.0,u0077,3.0,1.000000
78,0.0,u0079,2.0,1.000000
86,0.0,u0087,21.0,1.000000
93,0.0,u0094,1.0,1.000000
99,0.0,u0100,2.0,1.000000
122,0.0,u0123,1.0,1.000000
145,0.0,u0146,3.0,1.000000


In [54]:
sms_times_with_risk = sms_times_with_risk.drop(['label'], axis=1)
sms_times_with_risk

,uid,sms_count_with_risk,sms_fraction_with_risk
0,u0001,10.0,0.105263
1,u0002,0.0,0.000000
2,u0003,20.0,0.714286
3,u0004,21.0,0.169355
4,u0005,15.0,0.312500
5,u0006,21.0,0.700000
6,u0007,28.0,0.666667
7,u0008,16.0,0.042216
8,u0009,3.0,0.200000
9,u0010,7.0,0.205882


In [55]:
m = pd.Series(risk_voice_opp_number).isin(risk_sms_opp_number)
m.shape
m[m==True].shape

(28180,)

(2431,)

# Feat3

In [56]:
# 查看risk_sms_opp_number的opp_head, opp_len
sms_train[sms_train['opp_num'].isin(risk_sms_opp_number)]['opp_len'].value_counts()
sms_train[sms_train['opp_num'].isin(risk_sms_opp_number)]['opp_head'].value_counts()


13    70197
11    40533
9     12775
8      4789
7      3377
14     1418
10      951
5       492
12       86
15       10
16        7
17        5
20        4
18        4
4         2
2         2
19        1
6         1
Name: opp_len, dtype: int64

0      70154
130    27533
100    13757
106     5890
131     4077
861     3328
4       1388
139     1150
132      813
156      604
137      589
138      525
159      465
9        393
152      274
187      271
186      231
155      227
173      227
136      206
185      202
158      190
189      189
175      166
176      166
182      156
181      120
150      118
188      115
151      113
180      110
1        107
183      102
135      101
177      101
133       90
157       82
134       65
153       61
869       58
147       57
178       27
199       20
25         8
7          6
184        6
171        4
57         4
170        3
6          2
166        2
2          1
Name: opp_head, dtype: int64

In [57]:
a = sms_train[sms_train['uid'].isin(non_risk_uids_train)]

a[a['opp_len']==13][['opp_head','in_out']]
# a[a['opp_len']==13]['opp_head'].value_counts()
# a[a['opp_len']==13]['in_out'].value_counts()


,opp_head,in_out
1654,8,1
4429,8,1
14662,8,1
16662,8,1
19341,8,1
20382,6,1
23411,8,1
34486,8,1
34942,8,1
34944,8,1


# Feat4

In [58]:
sms_train['opp_num'].value_counts()
sms_train[sms_train['opp_num'].isin(risk_sms_opp_number)]['opp_num'].value_counts()


A7750EB1F3FB96632EE072A33943D3C6    70154
2C94BFAEEB30F858860967DC54844536    11511
1AD5351C43529967334D7CF8C4D0758A     3460
DE768264C1209BFD2FCBE076BC757FDA     3281
A3E288DF3A43B1486BBE768AFE9809F9     1489
046A6598F8BA90B15B8E5D991FBCB99B     1480
DCA1F127972B7B2FEAED5E7ABD88AC6B     1177
215778536180066660D0CC685667A9FB     1087
86F0652E76CA4DCE27D119B4D12FFF45     1086
3AC5268DE2D23E19C6FF326B291EA6BE     1012
A25F681038A570FC4330AE8CA3FFAD5B      949
3E634C402080AAE2DA0C20D61D80EDD5      901
3B9789F6DF67297C398E39C9D0E180CE      746
780EB73C10181C635A5F5B6A15805B63      741
E32316F1A0C19F32B6A42666B4A8E4DA      728
2CBE0ED988E2E09B689FFD983A57BF2C      703
FED69187DD999EE8A426FDA5256B2E71      688
6EC3B0399E83A13DCC3CF28E870E0C1D      658
50201D8941C6F842605A4FB007B16BB0      649
C55585C19667332B941925884B668D44      647
C8785E2C1212FCCA5C2CBC73A769DEEF      646
A3C818190441B8CE2DBBC07C81BEFA8A      629
AAAB5252F4CC1A98A7F8EFF43EB42B04      600
229C83DBC63B998E798DFD6BA51CF5CA  

A7750EB1F3FB96632EE072A33943D3C6    70154
2C94BFAEEB30F858860967DC54844536    11511
DE768264C1209BFD2FCBE076BC757FDA     3281
046A6598F8BA90B15B8E5D991FBCB99B     1480
86F0652E76CA4DCE27D119B4D12FFF45     1086
3E634C402080AAE2DA0C20D61D80EDD5      901
E32316F1A0C19F32B6A42666B4A8E4DA      728
2CBE0ED988E2E09B689FFD983A57BF2C      703
6EC3B0399E83A13DCC3CF28E870E0C1D      658
56804E38D7FFCB498BAE0A8B9B403606      592
E9BDB941C2DAD12717A8AEE4749C65D0      572
C9D88059454050AECBDB4EFE0303DB4F      473
D708E3AD8D740AF23EAFD52FFE7514F9      464
C79C78609E82998E589AA067C6A9FFF4      306
C7863EDADD4D8898A141D88190F368F3      303
7DED7750A46E131F94EE7A65BC6A5208      274
DB99302B51D2BC312E85C62088BD2D5C      261
C76C52D5761BF82F42F29F265DF3A5B8      242
E7654AEA8F2F809910E705BEE87CC050      233
C29D8820A1F050954B3B2989363D63A6      210
87992974430BC182E167395FE78E5B72      197
34C0B5D4A67CD59F6B2DC5EB81F0338B      194
242DF5CE6F84D7F38F9E6D3F9F54CBD6      177
F65A099EF78FFA6CB0680F1827839DD7  

可以发现，用户与有些号码之间发送或接受了大量短信

In [59]:
a = sms_train[sms_train['opp_num']== 'A7750EB1F3FB96632EE072A33943D3C6']['uid']
uid_train[uid_train['uid'].isin(a)]['label'].value_counts()


0    1472
1     687
Name: label, dtype: int64

# Feat5

In [60]:
sms_train['opp_num'].value_counts()

A7750EB1F3FB96632EE072A33943D3C6    70154
2C94BFAEEB30F858860967DC54844536    11511
1AD5351C43529967334D7CF8C4D0758A     3460
DE768264C1209BFD2FCBE076BC757FDA     3281
A3E288DF3A43B1486BBE768AFE9809F9     1489
046A6598F8BA90B15B8E5D991FBCB99B     1480
DCA1F127972B7B2FEAED5E7ABD88AC6B     1177
215778536180066660D0CC685667A9FB     1087
86F0652E76CA4DCE27D119B4D12FFF45     1086
3AC5268DE2D23E19C6FF326B291EA6BE     1012
A25F681038A570FC4330AE8CA3FFAD5B      949
3E634C402080AAE2DA0C20D61D80EDD5      901
3B9789F6DF67297C398E39C9D0E180CE      746
780EB73C10181C635A5F5B6A15805B63      741
E32316F1A0C19F32B6A42666B4A8E4DA      728
2CBE0ED988E2E09B689FFD983A57BF2C      703
FED69187DD999EE8A426FDA5256B2E71      688
6EC3B0399E83A13DCC3CF28E870E0C1D      658
50201D8941C6F842605A4FB007B16BB0      649
C55585C19667332B941925884B668D44      647
C8785E2C1212FCCA5C2CBC73A769DEEF      646
A3C818190441B8CE2DBBC07C81BEFA8A      629
AAAB5252F4CC1A98A7F8EFF43EB42B04      600
229C83DBC63B998E798DFD6BA51CF5CA  

In [61]:
# 将数据中的时间都换算为秒
def time2Second(time):
    seconds = int(time[0:2])*24*3600 + int(time[2:4])*3600 + int(time[4:6])*60 + int(time[6:8])
    return seconds

# voice['start_time'] = voice['start_time'].apply(time2Second)
# voice['end_time'] = voice['end_time'].apply(time2Second)
sms_train['start_time_number'] = sms_train['start_time'].apply(time2Second)
# sms_train.groupby('opp_num').apply(lambda x: x.sort_values(by=['start_time_number']))
sms_train['timediff'] = sms_train['start_time_number']
sms_train = sms_train.sort_values(by=['start_time_number'])
sms_train['timediff'] = sms_train['timediff'].diff(1)
sms_train[sms_train['opp_num'] == 'A7750EB1F3FB96632EE072A33943D3C6']

,uid,opp_num,opp_head,opp_len,start_time,in_out,start_time_number,timediff
239159,u1924,A7750EB1F3FB96632EE072A33943D3C6,0,13,01000317,1,86597,18.0
238980,u1154,A7750EB1F3FB96632EE072A33943D3C6,0,13,01002901,1,88141,267.0
270889,u4854,A7750EB1F3FB96632EE072A33943D3C6,0,13,01003006,1,88206,65.0
270888,u4854,A7750EB1F3FB96632EE072A33943D3C6,0,13,01003006,1,88206,0.0
278807,u3718,A7750EB1F3FB96632EE072A33943D3C6,0,13,01003037,1,88237,31.0
302763,u4143,A7750EB1F3FB96632EE072A33943D3C6,0,13,01003149,1,88309,1.0
273434,u4709,A7750EB1F3FB96632EE072A33943D3C6,0,13,01011629,1,90989,406.0
273433,u4709,A7750EB1F3FB96632EE072A33943D3C6,0,13,01011629,1,90989,0.0
148475,u4575,A7750EB1F3FB96632EE072A33943D3C6,0,13,01013336,1,92016,420.0
148474,u4575,A7750EB1F3FB96632EE072A33943D3C6,0,13,01013336,1,92016,0.0


# Feat 6

In [20]:
# 将数据中的时间提取出来
def time2dd(time):
    dd = int(time[0:2])
    return dd
def time2hh(time):
    hh = int(time[2:4])
    return hh
def time2mm(time):
    mm = int(time[4:6])
    return mm

voice['start_dd'] = voice['start_time'].apply(time2dd)
voice['start_hh'] = voice['start_time'].apply(time2hh)
voice['start_mm'] = voice['start_time'].apply(time2mm)

voice['end_dd'] = voice['end_time'].apply(time2dd)
voice['end_hh'] = voice['end_time'].apply(time2hh)
voice['end_mm'] = voice['end_time'].apply(time2mm)

sms['start_dd'] = sms['start_time'].apply(time2dd)
sms['start_hh'] = sms['start_time'].apply(time2hh)
sms['start_mm'] = sms['start_time'].apply(time2mm)


In [21]:
# 统计用户每天的通话数据
voice_per_day = voice.groupby(['uid', 'start_dd'])['opp_num'].count().reset_index()
voice_per_day = voice_per_day.groupby('uid')['opp_num'].agg(['std','max','min','median','mean','sum']).add_prefix('voice_per_day_').reset_index()

In [22]:
voice_per_day = voice_per_day.fillna(0)
pd.isnull(voice_per_day).any()
voice_per_day

uid                     False
voice_per_day_std       False
voice_per_day_max       False
voice_per_day_min       False
voice_per_day_median    False
voice_per_day_mean      False
voice_per_day_sum       False
dtype: bool

,uid,voice_per_day_std,voice_per_day_max,voice_per_day_min,voice_per_day_median,voice_per_day_mean,voice_per_day_sum
0,u0001,1.840991,8,1,2.0,2.548387,79
1,u0002,0.000000,1,1,1.0,1.000000,2
2,u0003,0.793200,4,1,1.0,1.312500,21
3,u0004,4.279133,23,1,5.0,6.195122,254
4,u0005,4.596338,23,1,8.0,9.113636,401
5,u0006,1.010712,4,1,1.0,1.692308,44
6,u0007,1.843453,7,1,3.0,2.885714,101
7,u0008,2.801766,12,1,5.0,5.318182,234
8,u0009,2.392917,11,1,1.0,2.742857,96
9,u0010,4.969422,27,1,4.0,4.814815,130


In [23]:
# 统计用户每天的短信数据
sms_per_day = sms.groupby(['uid', 'start_dd'])['opp_num'].count().reset_index()
sms_per_day = sms_per_day.groupby('uid')['opp_num'].agg(['std','max','min','median','mean','sum']).add_prefix('sms_per_day_').reset_index()

In [24]:
sms_per_day = sms_per_day.fillna(0)
pd.isnull(sms_per_day).any()
sms_per_day

uid                   False
sms_per_day_std       False
sms_per_day_max       False
sms_per_day_min       False
sms_per_day_median    False
sms_per_day_mean      False
sms_per_day_sum       False
dtype: bool

,uid,sms_per_day_std,sms_per_day_max,sms_per_day_min,sms_per_day_median,sms_per_day_mean,sms_per_day_sum
0,u0001,3.354102,14,1,3.0,3.800000,95
1,u0002,0.000000,1,1,1.0,1.000000,1
2,u0003,0.833809,4,1,2.0,1.866667,28
3,u0004,2.872611,12,1,3.0,3.757576,124
4,u0005,1.441064,6,1,1.0,1.920000,48
5,u0006,1.200490,6,1,2.0,1.764706,30
6,u0007,1.087811,5,1,2.5,2.625000,42
7,u0008,18.570220,123,1,6.0,9.023810,379
8,u0009,1.069045,4,1,2.0,2.142857,15
9,u0010,0.801315,4,1,2.0,1.700000,34


# Feat 7

In [25]:
# 统计用户与每个电话号码通话，短信的平均次数
voice_per_opp_num = voice.groupby(['uid', 'opp_num'])['opp_len'].count().reset_index()
voice_per_opp_num = voice_per_opp_num.groupby('uid')['opp_len'].agg(['std','max','min','median','mean','sum']).add_prefix('voice_per_opp_num_').reset_index()
voice_per_opp_num = voice_per_opp_num.fillna(0)

sms_per_opp_num = sms.groupby(['uid', 'opp_num'])['opp_len'].count().reset_index()
sms_per_opp_num = sms_per_opp_num.groupby('uid')['opp_len'].agg(['std','max','min','median','mean','sum']).add_prefix('sms_per_opp_num_').reset_index()
sms_per_opp_num = sms_per_opp_num.fillna(0)

In [26]:
pd.isnull(voice_per_opp_num).any()
voice_per_opp_num

pd.isnull(sms_per_opp_num).any()
sms_per_opp_num

uid                         False
voice_per_opp_num_std       False
voice_per_opp_num_max       False
voice_per_opp_num_min       False
voice_per_opp_num_median    False
voice_per_opp_num_mean      False
voice_per_opp_num_sum       False
dtype: bool

,uid,voice_per_opp_num_std,voice_per_opp_num_max,voice_per_opp_num_min,voice_per_opp_num_median,voice_per_opp_num_mean,voice_per_opp_num_sum
0,u0001,4.542488,16,1,1.0,3.590909,79
1,u0002,0.000000,1,1,1.0,1.000000,2
2,u0003,1.549193,7,1,1.0,1.400000,21
3,u0004,6.013286,41,1,1.0,3.298701,254
4,u0005,11.438520,67,1,3.0,7.290909,401
5,u0006,2.188716,10,1,1.0,2.095238,44
6,u0007,1.863946,10,1,1.0,1.683333,101
7,u0008,8.431903,69,1,1.0,3.295775,234
8,u0009,5.590809,22,1,2.0,4.571429,96
9,u0010,4.537630,21,1,2.0,3.513514,130


uid                       False
sms_per_opp_num_std       False
sms_per_opp_num_max       False
sms_per_opp_num_min       False
sms_per_opp_num_median    False
sms_per_opp_num_mean      False
sms_per_opp_num_sum       False
dtype: bool

,uid,sms_per_opp_num_std,sms_per_opp_num_max,sms_per_opp_num_min,sms_per_opp_num_median,sms_per_opp_num_mean,sms_per_opp_num_sum
0,u0001,13.691404,41,1,2.0,8.636364,95
1,u0002,0.000000,1,1,1.0,1.000000,1
2,u0003,8.485281,20,8,14.0,14.000000,28
3,u0004,24.806809,81,1,2.0,12.400000,124
4,u0005,13.686977,31,2,7.5,12.000000,48
5,u0006,9.539392,20,1,9.0,10.000000,30
6,u0007,9.899495,28,14,21.0,21.000000,42
7,u0008,73.002968,206,1,3.5,37.900000,379
8,u0009,6.363961,12,3,7.5,7.500000,15
9,u0010,14.142136,27,7,17.0,17.000000,34


# Feat 8

A concrete example of the entropy is the IP entropy: N!/(N_{IP1}! N{IP2}!... N{IPn}!). N is the total number of bids and N_{IPn} is the total number of bids placed from the nth IP. (Because the entropy is a large number, I used the log of the entropy.) The entropy is a measure of how both randomly distributed the bids are, how many IPs there are and how many bids there are. A bidder that places all their bids from the same IP has an entropy of N!/N! = 1.



In [27]:
# 计算用户通话记录的对端号码entropy
from collections import defaultdict

def calculate_entropy(group):
    opp_nums = defaultdict(int)
    
    for i, row in group.iterrows():
        opp_nums[row['opp_num']] += 1
    
    count = 0
    s = 1
    for key in opp_nums.keys():
        s = s * np.math.factorial(opp_nums[key])
        count += opp_nums[key]
    
    entropy = math.log(np.math.factorial(count)//s)
    return entropy
        


voice_opp_num_entropy = voice.groupby('uid').apply(calculate_entropy)
voice_opp_num_entropy = voice_opp_num_entropy.reset_index()
voice_opp_num_entropy.columns = ['uid', 'voice_opp_num_entropy']

In [28]:
voice_opp_num_entropy

,uid,voice_opp_num_entropy
0,u0001,173.184254
1,u0002,0.693147
2,u0003,36.854978
3,u0004,813.593310
4,u0005,1231.294234
5,u0006,95.891205
6,u0007,314.066713
7,u0008,682.179010
8,u0009,211.188296
9,u0010,358.284005


In [29]:
# 计算用户短信记录的对端号码entropy
sms_opp_num_entropy = sms.groupby('uid').apply(calculate_entropy)
sms_opp_num_entropy = sms_opp_num_entropy.reset_index()
sms_opp_num_entropy.columns = ['uid', 'sms_opp_num_entropy']

In [ ]:
# # 计算用户上网记录的网站名字（wa_name)的entropy
# def calculate_entropy_2(group):
#     opp_nums = defaultdict(int)
    
#     for i, row in group.iterrows():
#         opp_nums[row['wa_name']] += 1
    
#     count = 0
#     s = 1
#     for key in opp_nums.keys():
#         s = s * np.math.factorial(opp_nums[key])
#         count += opp_nums[key]
    
#     entropy = math.log(np.math.factorial(count)//s)
#     return entropy

# wa_name_entropy = wa.groupby('uid').apply(calculate_entropy_2)
# sms_opp_num_entropy = sms_opp_num_entropy.reset_index()
# sms_opp_num_entropy.columns = ['uid', 'sms_opp_num_entropy']

# Feat 9

In [7]:
voice[voice['opp_head'] == 7]['opp_len']

82278      13
263999     13
291836     13
299524     13
311379     13
346318     13
383899     13
383900     13
398941     13
398942     13
406479     13
413857     13
435768     13
439290     13
456249     13
514550     13
524154     13
527183     13
548439     13
565925     13
572591     13
572592     13
601437     13
601438     13
601439     13
628334     13
628335     13
642505     13
1058616    14
89005      14
251654     14
429329     14
605944     14
Name: opp_len, dtype: int64

In [9]:
voice['opp_head'].unique()

array([132, 139, 185, 100, 131, 166, 1, 130, 135, 138, 150, 189, 51, 133,
       186, 159, 158, 151, 153, 187, 184, 183, 182, 152, 180, 57, 177,
       157, 136, 181, 147, 188, 137, 29, 10, 21, 173, 156, 854, 134, 861,
       8, 25, 9, 155, 198, 176, 681, 43, 175, 178, 169, 836, 52, 83, 400,
       59, 828, 55, 873, 865, 845, 170, 171, 75, 53, 866, 95, 24, 833,
       832, 37, 39, 73, 848, 853, 852, 588, 79, 76, 20, 878, 91, 101, 116,
       22, 528, 862, 31, 843, 27, 199, 874, 805, 87, 885, 71, 834, 872,
       868, 851, 821, 879, 808, 835, 526, 669, 589, 125, 2, 527, 888, 857,
       179, 687, 846, 952, 876, 69, 4, 683, 886, 85, 6, 951, 838, 660,
       524, 74, 950, 855, 869, 639, 88, 889, 887, 867, 28, 666, 40, 895,
       856, 844, 522, 585, 56, 837, 576, 11, 826, 890, 521, 839, 93, 96,
       969, 842, 5, 23, 875, 361, 33, 672, 82, 884, 882, 523, 827, 850,
       859, 573, 823, 807, 822, 45, 17, 883, 877, 570, 577, 831, 7, 880,
       77, 81, 652, 689, 35, 566, 871, 89, 803, 863,

In [11]:
def int2Str(group):
#     print(group, type(group))
    res = str(group[0])
    
    if len(res) == 1 and group[1] > 5:
        res = '00' + res
    elif len(res) == 2:
        res = '0' + res
    
    return res

voice['opp_head'] = voice[['opp_head', 'opp_len']].apply(int2Str, axis=1)

In [14]:
voice['opp_head'].unique()

array(['132', '139', '185', '100', '131', '166', '1', '130', '135', '138',
       '150', '189', '051', '133', '186', '159', '158', '151', '153',
       '187', '184', '183', '182', '152', '180', '057', '177', '157',
       '136', '181', '147', '188', '137', '029', '010', '021', '173',
       '156', '854', '134', '861', '008', '025', '9', '155', '198', '176',
       '681', '043', '175', '178', '169', '836', '052', '083', '400',
       '059', '828', '055', '873', '001', '865', '845', '170', '171',
       '075', '053', '866', '095', '024', '009', '833', '832', '037',
       '039', '073', '848', '853', '852', '588', '079', '076', '020',
       '878', '091', '101', '116', '022', '528', '862', '031', '843',
       '027', '199', '874', '805', '087', '885', '071', '834', '872',
       '868', '851', '821', '879', '808', '835', '526', '669', '589',
       '125', '002', '527', '888', '857', '179', '687', '846', '952',
       '876', '069', '004', '683', '886', '085', '006', '951', '838',
       '66

In [15]:
voice_opp_head_one_hot = voice.groupby(['uid', 'opp_head'])['opp_head'].count().unstack().add_prefix('voice_opp_head_').reset_index().fillna(0)

In [16]:
voice_opp_head_one_hot

opp_head,uid,voice_opp_head_0,voice_opp_head_001,voice_opp_head_002,voice_opp_head_003,voice_opp_head_004,voice_opp_head_005,voice_opp_head_006,voice_opp_head_007,voice_opp_head_008,...,voice_opp_head_950,voice_opp_head_951,voice_opp_head_952,voice_opp_head_953,voice_opp_head_955,voice_opp_head_961,voice_opp_head_962,voice_opp_head_968,voice_opp_head_969,voice_opp_head_DDD
0,u0001,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,u0002,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,u0003,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,u0004,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,2.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,u0005,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
5,u0006,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
6,u0007,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
7,u0008,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
8,u0009,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
9,u0010,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


# Feat 10

In [17]:
sms_opp_head_one_hot = sms.groupby(['uid', 'opp_head'])['opp_head'].count().unstack().add_prefix('sms_opp_head_').reset_index().fillna(0)

In [35]:
len(sms['opp_head'].unique())

75

# 拼接

In [30]:
feat_all = uid

features = [voice_per_day, sms_per_day, voice_per_opp_num,
    sms_per_opp_num, voice_opp_num_entropy, sms_opp_num_entropy, voice_opp_head_one_hot]

for feat in features:
    feat_all = pd.merge(feat_all, feat, how='left', on='uid').fillna(0)

feat_all.to_csv('../myFeatures/feat_all_2.csv')

In [33]:
feat_all = pd.merge(feat_all, sms_opp_head_one_hot, how='left', on='uid').fillna(0)

In [34]:
train_features = feat_all[feat_all['uid'].isin(uid_train['uid'])]
test_features = feat_all[feat_all['uid'].isin(uid_test['uid'])]
train_features.shape
test_features.shape

(4999, 504)

(3000, 504)

In [36]:
train_features.to_csv('../myFeatures/train_features_2.csv',index=None)
test_features.to_csv('../myFeatures/test_features_2.csv',index=None)